# NeRF-COLAB

### [NeRF 공식 저장소](https://github.com/bmild/nerf)의 데모 실행 및 커스텀 데이터셋에 NeRF 모델 학습

Plank-ing Hyundong 3D Reconstruction Project
Created 2022.07.05 <br>

**NoteBook Author** <br>
[Janghoo Lee](https://www.linkedin.com/in/janghoo-lee-25212a1a0) <br>
🎓 : [ProtossDragoon](https://github.com/ProtossDragoon) <br>
📧 : dlwkdgn3@gmail.com <br>

🤔 If you have any questions, please raise an issue in our [github repo](https://github.com/ProtossDragoon/PlankHyundong).

## 환경

### TF 런타임 변경

In [ ]:
%tensorflow_version 1.x

### 의존 패키지 설치

In [ ]:
!sudo apt -qq install imagemagick
!pip install ConfigArgParse -qqq
!pip install imageio-ffmpeg -qqq

### 공식 저장소

In [ ]:
!git clone https://github.com/bmild/nerf.git

In [ ]:
%cd nerf
!ls -al

## 텐서보드

In [ ]:
%load_ext tensorboard

**텐서보드 해석**

output 함수의 리턴값 [참고1](https://github.com/bmild/nerf/blob/18b8aebda6700ed659cb27a0c348b737a5f6ab60/run_nerf.py#L284-L287), [참고2](https://github.com/bmild/nerf/blob/18b8aebda6700ed659cb27a0c348b737a5f6ab60/run_nerf.py#L81-L89)을 먼저 해석해 보자.
- rgb_map: [batch_size, 3]. Predicted RGB values for rays.
- disp_map: [batch_size]. Disparity map. Inverse of depth.
- acc_map: [batch_size]. Accumulated opacity (alpha) along a ray. (Ray 위의 모든 지점의 투명도값들을 전부 더한 결과. Transmittance 라고 볼 수도 있음. 뭐 어쨌든, 값이 높을수록(float 표현의 경우 1에 가까울수록) 투명하지 않은 것임)
- extras: dict with everything returned by render_rays().

**Coarse, Fine**: NeRF 논문을 보면 알겠지만, Coarse 모드로 한번 학습을 시킨 다음에 Fine 모드로 한번 학습을 다시 시키는 구조로 이루어져 있다. 자세한 내용은 논문을 참조.

TensorBoard IMAGES 탭을 보자
- rgb: Coarse 모드의 rgb_map 에 대응됨
- disp: Coarse 모드의 disp_map 에 대응됨
- acc: Coarse 모드의 acc_map 에 대응됨
- rgb0: Fine 모드의 rgb_map 에 대응됨
- disp0: Fine 모드의 disp_map 에 대응됨
- acc0: Fine 모드의 acc_map 에 대응됨
- rgb_holdout: rgb_map 이 생성해야 하는 영상의 정답값임
- z_std: [num_rays]. Standard deviation of distances along ray for each sample.

In [ ]:
%tensorboard --logdir ./logs

## 빠른 데모 실행

### 샘플 데이터 다운로드

In [ ]:
!bash download_example_data.sh

### 샘플 데이터로 NeRF 실행

In [ ]:
!python run_nerf.py --config config_fern.txt

In [ ]:
!python run_nerf.py --config config_lego.txt

## 커스텀 데이터로 실행

- LLFF 를 통해 얻은 pose 가 필요합니다.

### 환경

In [ ]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

In [ ]:
from datetime import datetime
now = datetime.now().strftime('%y%m%d_%H%M%S')

dataset_name = 'steeltable' #@param ['toilet', 'napkin', 'woodtable', 'steeltable']
downsample_factor = 32 #@param {type:"slider", min:8, max:64, step:8}
netdepth = 4 #@param {type:"slider", min:4, max:8, step:1}
netwidth = 64 #@param {type:"slider", min:64, max:256, step:4}
n_rays = 131072 #@param {type:"slider", min:1024, max:131072, step:32}
n_pts = 262144 #@param {type:"slider", min:2048, max:262144, step:64}
experiment_name = f'{dataset_name}_{downsample_factor}_downsampled_{now}'
lr = 1e-4

_dummy_dir = f'./logs/{experiment_name}'
_tensorboard_logdir = f'./logs/summaries/{experiment_name}'
# !rm -r {_dummy_dir}
# !rm -r {_tensorboard_logdir}
print(f'experiment: {experiment_name}')

### 실행

**플래그 해석**

- no_ndc flag: do not use normalized device coordinates
- spherify: set for spherical 360 scenes
- lindisp: sampling linearly in disparity rather than depth


**실행하기 전에**

- `nerf/run_nerf.py` 의 [746번째 줄](https://github.com/bmild/nerf/blob/18b8aebda6700ed659cb27a0c348b737a5f6ab60/run_nerf.py#L746)에 있는 `N_iters = 1000000` 을 변경해 주어야 합니다. COLAB PRO+ 런타임이 아닌 경우, 런타임이 학습이 마칠 때까지 기다려주지 못하기 때문입니다. 필자는 COLAB PRO GPU 에서 2~4시간정도면 학습이 종료되는 100,000 정도를 추천합니다.

In [ ]:
!python run_nerf.py \
    --datadir /content/drive/MyDrive/dev/llff_data/{dataset_name} \
    --dataset_type llff \
    --factor {downsample_factor} \
    --netdepth {netdepth}\
    --netwidth {netwidth} \
    --netdepth_fine {netdepth} \
    --netwidth_fine {netwidth} \
    --chunk {n_rays} \
    --netchunk {n_pts} \
    --lrate {lr} \
    --expname {experiment_name}
# --no_ndc
# --spherify
# --lindisp

## 실험 관리 도구

- 실험 관리 도구로 wandb 를 사용합니다.
- [plank-hyundong](https://wandb.ai/plank-hyundong) 팀의 [wandb 프로젝트 plank-hyundong](https://wandb.ai/plank-hyundong/plank-hyundong) 관리자에게 접근 키를 문의하세요.

In [ ]:
!pip install wandb -qqq

In [ ]:
#!wandb login --relogin

In [ ]:
# Log in to your W&B account
import wandb
wandb.login()

In [ ]:
# Start a new run to track this script
wandb.init(
    # Set the project where this run will be logged
    project="plank-hyundong",
    entity="plank-hyundong",
    # We pass a run name (otherwise it’ll be randomly assigned, like sunshine-lollypop-10)
    name=f'{experiment_name}'
)

In [ ]:
import os
import pprint
import glob
import pathlib

p = os.path.join(_dummy_dir, 'args.txt')
config = {}
with open(p, 'r') as f:
    for line in f.readlines():
        # 공백을 제거하고 = 을 기준으로 키와 값으로 분리
        k, v = line.replace(' ', '').replace('\n', '').split('=')
        # 딕셔너리 형태로 저장
        config[k] = v
    pprint.pprint(config)

# Track hyperparameters and run metadata
wandb.config.update(config)

In [ ]:
result_video_pattern = os.path.join(_dummy_dir, f'{experiment_name}_*.mp4')
print(result_video_pattern)
paths = glob.glob(result_video_pattern)
pprint.pprint(paths)

for path in paths:
    # filename: {experiment_name}_spiral_{step}_{channel}.mp4
    _, step_channel = path.rstrip('.mp4').split('spiral_')
    step, channel = step_channel.split('_')
    wandb.log({
        f'video_{channel}':wandb.Video(path, fps=15, format="mp4"),
    }, step=int(step))

In [ ]:
val_image_pattern = os.path.join(_dummy_dir, 'tboard_val_imgs', '*')
print(val_image_pattern)
paths = glob.glob(val_image_pattern)
pprint.pprint(paths)

for path in paths:
    # filename: {step}.png
    step = os.path.basename(path).rstrip('.png')
    wandb.log({
        f'image_val':wandb.Image(path),
    }, step=int(step))

In [ ]:
test_image_pattern = os.path.join(_dummy_dir, 'testset_*')
print(test_image_pattern)
paths = glob.glob(test_image_pattern)
pprint.pprint(paths)

for path in paths:
    # foldername: testset_{step}
    _, step = pathlib.PurePath(path).name.split('_')
    for (dir_path, dir_names, img_names) in os.walk(path):
        for img_name in img_names:
            wandb.log({
                f'image_test_{img_name}': wandb.Image(os.path.join(dir_path, img_name)),
            }, step=int(step))